In [ ]:
from PIL import Image
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
from osgeo import gdal

def create_tiles(input_file, output_dir, tile_size_x, tile_size_y, overlap=0):
    input_ds = gdal.Open(input_file)
    cols = input_ds.RasterXSize
    rows = input_ds.RasterYSize

    for i in range(0, cols, tile_size_x - overlap):
        for j in range(0, rows, tile_size_y - overlap):
            output_file = os.path.join(output_dir, f"tile_{i}_{j}.tif") #here
            gdal.Translate(output_file, input_ds, srcWin=[i, j, tile_size_x, tile_size_y])

    input_ds = None

def show_images(images, titles=None):
    if not titles:
        titles = [img.shape for img in images]
    fig, axes = plt.subplots(nrows=1, ncols=len(images), figsize=(10, 30))
    for i, ax in enumerate(axes):
        ax.imshow(images[i], cmap="summer")
        ax.set_title(titles[i])
        ax.axis("off")
    plt.show()

def show_1image(image, size = (10,10)):
    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=size)
    axes.imshow(image, cmap="summer")
    axes.axis("off")
    plt.show()

def del_dir_contents(dir_path):
  for file in os.listdir(dir_path):
    file_path = os.path.join(dir_path, file)
    os.remove(file_path)


# Directory Setup
Set paths according to where the files are located

In [ ]:
# Input
dir_parent = '/content'
dir_in = '/content/drive/MyDrive/SeniorProjectMockup/input'
file_name = 'bkk1_02.tif'

# Output
dir_tiles = '/content/tiles'
dir_masks = '/content/masks'

# Image dimensions
dim = 512

file_path = os.path.join(dir_in,file_name)

Delete contents of the Tile folder to have a fresh start

In [ ]:
del_path = dir_tiles
try:
  del_dir_contents(del_path)
except:
  pass

Create tile folder and tiles from the input image (file_path)

In [ ]:
try:
  os.mkdir(dir_tiles)
except:
  pass

create_tiles(file_path, dir_tiles, dim, dim, overlap=0)

Delete contents of the Mask folder to have a fresh start

In [ ]:
del_path = dir_masks
try:
  del_dir_contents(del_path)
except:
  pass

Create Mask folder

In [ ]:
try:
  os.mkdir(dir_masks)
except:
  pass

# Model Setup

In [ ]:
!pip install -U -q segmentation-models
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

In [ ]:
backbone = 'inceptionresnetv2'

In [ ]:
backbone_preprocess = sm.get_preprocessing(backbone)
preprocess_fn = lambda x, y: (backbone_preprocess(x), y)

Set model_path according to where the model is located

In [ ]:
model_path = "/content/drive/MyDrive/SeniorProjectMockup/models/model_unet-inceptionresnetv2_T455_PT.keras"
model = keras.models.load_model(
    model_path,
    custom_objects={
        "dice_loss": sm.losses.DiceLoss(),
        "iou_score": sm.metrics.IOUScore()
    },
)

# Model Showcase

In [ ]:
new_size = (512, 512)
threshold = 0.9
tiles_left = len(os.listdir(dir_tiles))

for tile in os.listdir(dir_tiles):
  img_path = os.path.join(dir_tiles, tile)

  src_ds = gdal.Open(img_path)

  image = plt.imread(img_path)
  image = image[:,:,:3]
  image = image.astype(int)
  #image = tf.image.resize(image, new_size)
  processed_test_img = backbone_preprocess(image)

  mask_4d = model.predict(np.expand_dims(processed_test_img, axis=(0)))
  pred_mask_prob = np.squeeze(mask_4d, axis=0)
  pred_mask = np.where(pred_mask_prob > threshold, 1, 0)

  tile = tile.split('.')
  mask_path = os.path.join(dir_masks,tile[0]+'_mask.'+tile[1])
  cv2.imwrite(mask_path, pred_mask)

  target_ds = gdal.Open(mask_path, gdal.GA_Update)
  target_ds.SetGeoTransform(src_ds.GetGeoTransform())
  target_ds.SetProjection(src_ds.GetProjection())
  src_ds = None
  target_ds = None

  show_images((image, pred_mask), [tile[0], "Model Prediciton"])
  tiles_left -= 1
  print("tiles left",tiles_left,"...")


# Merge mask tiles into one

In [ ]:
def merge_tiles(input_dir, output_file):
    # Get list of tile files
    tile_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.tif')]

    # Build VRT from tile files
    vrt_options = gdal.BuildVRTOptions(resampleAlg='nearest')
    vrt = gdal.BuildVRT(os.path.join(output_file + '.vrt'), tile_files, options=vrt_options)

    try:
        # Translate VRT to final output file
        gdal.Translate(output_file, vrt)

    except ValueError as e:
        print("Error during translation:", e)
        print("Applying transparency to problematic sections.")

        # Get NoData value
        no_data_value = vrt.GetRasterBand(1).GetNoDataValue()

        # Set transparent pixels
        gdal.Translate(output_file, vrt, format='GTiff', creationOptions=['ALPHA=YES'], noData=no_data_value)

    finally:
        # Close datasets
        vrt = None



In [ ]:
input_dir = dir_masks
output_file = "merged_output.tif"
merge_tiles(input_dir, output_file)

In [ ]:
merged_mask = '/content/merged_output.tif'
image = Image.open(merged_mask)

# Convert the image to grayscale
gray_image = image.convert('L')

show_1image(gray_image)

In [ ]:
numpy_array = np.array(gray_image)
print(np.unique(numpy_array))

# Extract LatLongBound
This step is used to extract the values used for overlaying the image on Google Map Javascript API

In [ ]:
from osgeo import gdal, osr
from typing import Tuple

def get_geotiff_bounds(file_path: str) -> Tuple[float, float, float, float]:
    """
    Get the bounding box (in WGS84 coordinates) of a GeoTIFF file.

    Args:
        file_path (str): Path to the GeoTIFF file.

    Returns:
        Tuple[float, float, float, float]: Bounding box coordinates in the order (min_lng, min_lat, max_lng, max_lat).
    """
    # Open the GeoTIFF file
    dataset = gdal.Open(file_path)

    # Reproject to WGS84
    target_srs = osr.SpatialReference()
    target_srs.ImportFromEPSG(4326)  # WGS84 EPSG code
    options = gdal.WarpOptions(dstSRS=target_srs)
    gdal.Warp("/vsimem/temp.tif", dataset, options=options)

    # Open the reprojected file
    reprojected_dataset = gdal.Open("/vsimem/temp.tif")

    # Get geotransform information
    geotransform = reprojected_dataset.GetGeoTransform()

    # Get raster dimensions
    width = reprojected_dataset.RasterXSize
    height = reprojected_dataset.RasterYSize

    # Calculate bounding box coordinates
    min_lng = geotransform[0]
    max_lng = geotransform[0] + width * geotransform[1] + height * geotransform[2]
    max_lat = geotransform[3]
    min_lat = geotransform[3] + width * geotransform[4] + height * geotransform[5]

    # Close the reprojected dataset and delete the temporary file
    reprojected_dataset = None
    gdal.Unlink("/vsimem/temp.tif")

    return min_lng, min_lat, max_lng, max_lat


In [ ]:
file_path = merged_mask
bounds = get_geotiff_bounds(file_path)
print("Bounding box coordinates (min_lng(west), min_lat(south), max_lng(east), max_lat(north)):", bounds)

# Raster to KLM

In [ ]:
!pip install -q simplekml
!pip install -q rasterio

In [ ]:
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape, mapping
import simplekml
import pyproj

def mask_to_kml(mask_path, kml_path, mask_value=1):
    """
    Converts a GeoTIFF mask raster to a KML file, highlighting areas with a specific mask value
    and including latitude and longitude information.

    Args:
        mask_path (str): Path to the GeoTIFF mask raster.
        kml_path (str): Path to save the generated KML file.
        mask_value (int, optional): Value in the mask raster representing the areas to highlight. Defaults to 1.
    """

    with rasterio.open(mask_path) as src:
        image = src.read(1)  # Read the first band (assuming single-band mask)
        transform = src.transform

        # Get CRS and create transformer for lat/lon conversion
        crs = src.crs
        transformer = pyproj.Transformer.from_crs(crs, pyproj.CRS("EPSG:4326"), always_xy=True)

        # Extract shapes from mask
        results = (
            {'properties': {'raster_val': v}, 'geometry': s}
            for i, (s, v) in enumerate(shapes(image, transform=transform))
            if v == mask_value
        )

        # Create KML document
        kml = simplekml.Kml()
        fol = kml.newfolder(name="Mask Areas")  # Create a folder to organize polygons

        for result in results:
            geom = shape(result['geometry'])

            # Convert coordinates to lat/lon
            lat_lon_coords = []
            for x, y in geom.exterior.coords:
                lon, lat = transformer.transform(x, y)
                lat_lon_coords.append((lon, lat))

            pol = fol.newpolygon(name=f"Polygon {result['properties']['raster_val']}")
            pol.outerboundaryis = lat_lon_coords

    kml.save(kml_path)

In [ ]:
mask_raster_path = merged_mask
mask_kml_path = "mask.kml"
mask_to_kml(mask_raster_path, mask_kml_path)  # Use default mask_value=1

# Post proccesing and attribute extraction on KML

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pandas as pd
import fiona
import simplekml

def calculate_polygon_areas_and_export(kml_file, output_kml_file, tolerance=1, min_area=2.0):
    # Read KML file
    fiona.supported_drivers['KML'] = 'rw'
    gdf = gpd.read_file(kml_file, driver='KML')

    # Ensure it's a GeoDataFrame with polygons
    if 'geometry' not in gdf.columns or gdf.geom_type[0] != 'Polygon':
        raise ValueError("Input file does not contain polygon geometries.")

    # Simplify polygons using the Douglas-Peucker algorithm
    gdf = gdf.to_crs(epsg=24047)
    gdf['geometry'] = gdf['geometry'].simplify(tolerance, preserve_topology=True)

    # Calculate areas
    polygons = gdf[gdf.geometry.type == 'Polygon']

    # Change name of polygons to their size (Google map displays the polygon names when clicked)
    gdf['area'] = polygons.geometry.area
    gdf['Name'] = gdf['area'].round(2)

    gdf_min_area = gdf[gdf['area'] >= min_area]

    total_area = gdf_min_area['Name'].sum().round(2)

    gdf_min_area = gdf_min_area.to_crs(epsg=4326)

    print(gdf_min_area)
    print(total_area)

    # Export to KML
    kml = simplekml.Kml()
    for index, row in gdf_min_area.iterrows():
        poly = kml.newpolygon(name=str(row['Name'])+ " m^2", outerboundaryis=row['geometry'].exterior.coords[:])
        poly.style.linestyle.color = simplekml.Color.rgb(0, 133, 0)
        poly.style.linestyle.width = 2
        poly.style.polystyle.fill = 1
        poly.style.polystyle.color = simplekml.Color.changealphaint(200, simplekml.Color.rgb(0, 255, 0))

    kml.save(output_kml_file)
    print("KML file exported successfully.")

    return total_area

In [ ]:
tolerance=1 # unit: meter
min_area=2 # unit: meters squared
input_kml_file = "/content/mask.kml"
output_kml_file = "/content/mask_simplified.kml"

try:
  total_area = calculate_polygon_areas_and_export(input_kml_file, output_kml_file, tolerance, min_area)
except Exception as e:
  print('Polygons:', e) # no polygons


# Power Generation formula

In [ ]:
# User inputs
watt_per_m2 = 1000
hours_per_day = 5
effeciency = 0.15 # Average 15-20%

# Daily Output (kWh) = Wattage (W) x Hours of Sunlight x Efficiency
watt_hour = total_area * watt_per_m2
daily_output = watt_hour/1000 * hours_per_day * effeciency
print("Daily Output:",np.round(daily_output,2), "kWh")